In [ ]:
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.ipac.ned import Ned
import numpy as np
from astropy.cosmology import WMAP9 as cosmo
from urllib.request import urlretrieve
from collections import Counter
import matplotlib.pyplot as plt
import pickle
from astropy.io import fits
from astropy.table import Table
import glob
import os
from scipy.interpolate import CubicSpline
import textwrap
import seaborn as sns
import scipy.stats as scp

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Times"],
    "text.latex.preamble": r"\usepackage{amsmath}\usepackage{mathptmx}",  # Times Roman
    "hatch.linewidth": 3.0,
})
sns.set_context("paper",font_scale=1.75)


def make_desig(data, ra_key='ra', dec_key='dec'):
    """make designation if df has 'ra' and 'dec' columns"""
    desig=[]
    for posstring in SkyCoord(data[ra_key].values*u.deg, data[dec_key].values*u.deg).to_string("hmsdms"):
        posstring = posstring.split(' ')
        des_ra = posstring[0][0:2]+posstring[0][3:5]
        des_dec = posstring[1][0:3]+posstring[1][4:6]
        desig.append('J'+des_ra+des_dec)
    return desig

def pos(row):
    """make skyCoord object for HST coord cone search"""
    return SkyCoord(ra=row['RA']*u.deg, dec=row['DEC']*u.deg)

def load_bigmac():
    """crossmatch sample with big MAC"""
    # read in big  mac
    bigmac = pd.read_csv("/home/insepien/research-data/GFG.csv")
    # format designation
    desigs = []
    for i in range(len(bigmac)):
        name = bigmac['Name1'].loc[i].replace("SDSS","")
        if name[0] == "J":
            if "+" in name:
                desig = name.split("+")[0][:5] + "+" + name.split("+")[1][:4]
                desigs.append(desig)
            elif "-" in name:
                desig = name.split("-")[0][:5] + "-" + name.split("-")[1][:4]
                desigs.append(desig)
            else: print(name) 
        else:
            desigs.append(name)
    bigmac['Desig'] = desigs
    # optionally can get decals images
    # for n in mul_bm.index:
    #     urlretrieve('http://legacysurvey.org/viewer/jpeg-cutout?ra='+str(mul_bm.loc[n,'RA'])+'&dec='+str(mul_bm.loc[n,'DEC'])+'&layer=decals-dr7&pixscale=0.27&bands=grz',
    #                 "/home/insepien/research-data/hst/mul_bm/"+str(mul_bm.loc[n,'DESIG'])+'.jpg')
    return bigmac

def cal_sep(theta, z):
    """return dual sep in kpc given scalar angle sep in arcsec"""
    angle = (theta*u.arcsec).to(u.rad).value
    return (cosmo.angular_diameter_distance(z)*angle).to(u.kpc)


def f(on, theta):
    """plot decal image and annulus at detected dual separation theta"""
    fn = "/home/insepien/research-data/hst/mul_bm/"+on+".jpg"
    decals_plate_scale = 0.236 #''/pix
    pix_sep = theta/decals_plate_scale

    fig,ax = plt.subplots()
    im = plt.imread(fn)
    midF = im.shape[0]/2
    ax.imshow(im)
    circ = plt.Circle((midF,midF), pix_sep, fill=False, color='white',alpha=0.5,label=f"{theta:.2f}''")
    ax.add_patch(circ)
    ax.legend()
    ax.set_title(on)


load data

In [ ]:
alpaka = pd.read_pickle("/home/insepien/research-data/alpaka/alpaka_z05_merged_wise.pkl")
# alpaka_fullZ = Table(fits.getdata("/home/insepien/research-data/alpaka/ALPAKA_v1_withDes.fits")).to_pandas()
magel_o = pd.read_pickle("/home/insepien/research-data/alpaka/magellan/alpaka_39fits.pkl")
magel = alpaka[(alpaka['RA'].isin(magel_o['RA'])) & (alpaka['DEC'].isin(magel_o['DEC']))]
bigmac = load_bigmac()
# separate dual and singles
dualnames = ["J1215+1344","J1222-0007"]
singlenames =  magel[~ magel['Desig'].isin(dualnames)]['Desig']
duals = alpaka[alpaka['Desig'].isin(dualnames)]
singles = alpaka[alpaka['Desig'].isin(singlenames)]
print(duals.shape,singles.shape)

cross-match with Big MAC for paper intro

In [ ]:
# get some stats on sub-kpc pairs for science justification
not_recoil = bigmac['Primary System Type']!='Recoil Candidate'
subkpc_mask = not_recoil & (bigmac['Sep(kpc)']<1) & (bigmac['Sep(kpc)']>0.3)
kpc_mask = not_recoil & (bigmac['Sep(kpc)']>1)
dual_mask = subkpc_mask | kpc_mask
confirmed_mask = dual_mask & ~bigmac["Primary System Type"].str.contains("Candidate")

# fractions
print(f"fraction of sub-kpc/total dual = {subkpc_mask.sum()}/{dual_mask.sum()} = {subkpc_mask.sum()/dual_mask.sum():.3f}")
print(f"fraction of confirmed/total dual = {confirmed_mask.sum()/dual_mask.sum():.3f}")

# get some methods of measuring sub-kpc sep
anyl_meth = Counter(bigmac[subkpc_mask]['Parsed Analysis Method'])
print("1st most common method: ",anyl_meth.most_common(2)[0])
print("2nd most common method: ",anyl_meth.most_common(2)[1])

sample plot of IR only for methods section

In [ ]:
def norm_hist(ax,quant,ecolor,fcolor,bin_arr=[],alpha=1,horz=False,hatchsym=''):
    """normalize histogram sum count to 1 given some quantity (quant)
        args: edgecoloe, facecolor, bin array, opacity, flag for plotting horizontal hist"""
    count, bin = np.histogram(quant,bins=bin_arr)
    if horz:
        ax.barh(bin[:-1],count/np.sum(count), height= np.diff(bin),
                align='edge',edgecolor=ecolor,facecolor=fcolor,alpha=alpha,hatch=hatchsym,hatch_linewidth=0.5)
    else:
        ax.bar(bin[:-1],count/np.sum(count),width = np.diff(bin),
               align='edge',edgecolor=ecolor,facecolor=fcolor,alpha=alpha,hatch=hatchsym,hatch_linewidth=0.5)
        


fig,ax = plt.subplots(2,2,gridspec_kw={'width_ratios': [2,0.5],'height_ratios': [0.7,2]},figsize=(8,6),
                      sharey='row',sharex='col',dpi=200)
plt.subplots_adjust(wspace=0.05,hspace=0.05)

mdual_labs = ['J1215+1344','J1222-0007 W', 'J1222-0007 E']
mask171 = ((alpaka['OIII_5007_LUM_DERRED']*600 > 1e46) & (alpaka['Z']> 0.14) & (alpaka['Z']<0.22))

# scatter type-2 in z=0.1-0.5 with match in wise
ax[1,0].scatter(alpaka['Z'],np.log10(alpaka['irbol']),s=2,alpha=0.1,color="plum")
# same as above but Lbol OIII > 1e46
ax[1,0].scatter(alpaka['Z'][mask171],np.log10(alpaka['irbol'][mask171]),s=7,color="darkseagreen",marker="x")
# magellan sample
ax[1,0].scatter(singles['Z'],np.log10(singles['irbol']),s=50,marker='2',color='indigo',alpha=0.7)
[ax[1,0].scatter(duals['Z'].values[i],np.log10(duals['irbol'].values[i]),
                 s=40,marker=['s',"",'o'][i],color='indigo',alpha=0.7,label=mdual_labs[i]) for i in [0,2]];

ax[1,0].set_ylim(43.5,47.5)
ax[1,0].set_xlim(0.09,0.3)
ax[1,0].set_xlabel("Redshift")
ax[1,0].set_ylabel('Log($L_{\\rm{bol,~IR}}$) $[\\rm{erg~s}^-1]$')
ax[1,0].legend(fontsize=10) 

# hist z
binz = np.linspace(np.min(alpaka['Z']),np.max(alpaka['Z']),20)
norm_hist(ax[0,0],alpaka['Z'],"plum",'none',bin_arr=binz)
norm_hist(ax[0,0],alpaka['Z'][mask171],"darkseagreen",'none',bin_arr=binz,hatchsym="/")
norm_hist(ax[0,0],np.concatenate([singles['Z'],duals['Z']]),'none','indigo',bin_arr=binz,alpha=0.5)
ax[0,0].set_ylabel("Fraction")

# hist Lbol
binL = np.linspace(np.log10(np.min(alpaka['irbol'])), np.log10(np.max(alpaka['irbol'])),10)
norm_hist(ax[1,1],np.log10(alpaka['irbol'].dropna()),'plum',"none",binL,horz=True)
norm_hist(ax[1,1],np.log10(alpaka['irbol'].dropna()[mask171]),'darkseagreen',"none",binL,horz=True,hatchsym="/")
norm_hist(ax[1,1],np.log10(np.concatenate([singles['irbol'],duals['irbol']])),'none','indigo',binL,alpha=0.5,horz=True)
ax_top = ax[1,1].secondary_xaxis("top")
ax_top.set_xlabel("Fraction")
ax_top.set_xticks([0,0.2])
ax_top.set_xticklabels([0,0.2])
ax[1,1].set_xticks([])

ax[0,1].axis('off');

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,4),sharey=True,sharex=True,dpi=500)
ax[0].scatter(alpaka['Z'],np.log10(alpaka['OIII_5007_LUM_DERRED']*800),s=2,alpha=0.1,color='plum')
ax[0].scatter(magel['Z'],np.log10(magel['OIII_5007_LUM_DERRED']*800),s=50,alpha=0.5,marker="2",color='indigo')
mdual_labs = ['J1215+1344','J1222-0007 W', 'J1222-0007 E']
[ax[0].scatter(duals['Z'].values[i],np.log10(duals['OIII_5007_LUM_DERRED'].values*800)[i],s=40,marker=['s','o',"^"][i],color='indigo',alpha=0.5,label=mdual_labs[i]) for i in range(3)];

ax[1].scatter(alpaka['Z'],np.log10(alpaka['irbol']),s=2,alpha=0.1,color='plum')
ax[1].scatter(singles['Z'],np.log10(singles['irbol']),s=50,marker='2',color='indigo',alpha=0.5)
[ax[1].scatter(duals['Z'].values[i],np.log10(duals['irbol'].values)[i],s=40,marker=['s','o','^'][i],color='indigo',alpha=0.5,label=[mdual_labs[0],"",mdual_labs[2]][i]) for i in [0,2]];

ax[0].scatter(alpaka[j1010mask]['Z'],np.log10(alpaka[j1010mask]['OIII_5007_LUM_DERRED']*800),marker="x",color="k",label='J1010+1413')
ax[1].scatter(alpaka[j1010mask]['Z'],np.log10(alpaka[j1010mask]['irbol']),marker="x",color="k",label='J1010+1413')

ax[0].set_ylim(43,48)
ax[0].set_xlim(0.09,0.3)
[a.set_xlabel("Redshift") for a in ax]
[ax[i].set_ylabel(['Log($L_{\\rm{bol,~[OIII]~dered}}$) $[\\rm{erg~s}^-1]$','Log($L_{\\rm{bol,~IR}}$) $[\\rm{erg~s}^-1]$'][i]) for i in range(2)]
[ax[i].legend(fontsize=10) for i in range(2)]
[ax[i].text(0.025, 0.975, ["[OIII]","IR"][i], transform=ax[i].transAxes, ha='left', va='top',fontsize=20) for i in range(2)]

;

check emission line luminosities: J1010 is dimmer than rest of sample based on OIII dereddened, but is brighter than the rest in SDSS R-band. R-band covers OIII and HB, so it should be OIII dominated. Therefore OIII dereddened measurments do not make sense

In [ ]:
#### check emission lines
j10 = alpaka[alpaka['Desig'] == "J1010+1413"]
keys = ['HB_LUM',
 'OIII_4959_LUM',
 'OIII_5007_LUM',
 'NII_6548_LUM',
 'HA_LUM',
 'NII_6584_LUM',
 'NVS_LUM']
i=0
clr = sns.color_palette("colorblind", len(keys))
for k in keys:
    lums = [m for m in magel[k] if np.isfinite(m) and m!=0]
    plt.hist(np.log10(lums),color=clr[i],alpha=0.7,label=k)
    try:
        plt.axvline(np.log10(j10[k].values),c=clr[i])
    except:
        print(k)
    i+=1
plt.xlim((37.5,43.5))
plt.legend(bbox_to_anchor=(1,1))
plt.title("J1010 is generally brighter than magellan sample");

code below to make dual sep vs redshift plot (used for hst phase 1)

In [ ]:
from matplotlib.lines import Line2D
import matplotlib.patches as patches
sns.set_context("paper",font_scale=1.75)
sns.set_style('ticks')
sns.set_palette('colorblind')
figparams = {'font.family': 'DejaVu Sans',
            'font.serif':'Times',
            'hatch.linewidth' : 3.0}
plt.rcParams.update(figparams)

fig,ax = plt.subplots(figsize=(15,7),dpi=500)
# plot hst resolution limit
for reso in [1,5,20]:
    seeing = 0.04*2.5*reso
    ax.plot(np.linspace(0,3),cal_sep(seeing, np.linspace(0,3)),c='k',linestyle="--",alpha=0.5)
    ax.text(2.5, 1*reso, f"{seeing:.1f} arcsec", color='k',fontsize=15)

# add survey vol
# rect = patches.Rectangle((0.1, 0), 0.4-0.1, 1, linewidth=2, edgecolor='w', facecolor='darkseagreen',alpha=0.3)
# ax.add_patch(rect)
# ax.text(0.2,1,"Our survey",c="darkolivegreen",fontsize=15)

# kpc points ie sep>1 with confidence>0.5, z0-3, have been imaged
conf_kpc = dual['ST1 Confidence Flag']>=0.5
imag_kpc = dual['Parsed Analysis Method'].str.contains("Imaging")
z_mask = (dual['z1']>0)&(dual['z1']<3)
kpcdf = dual[conf_kpc&imag_kpc&z_mask]
paper = list(kpcdf['Paper(s)'].str.split(" ; ").explode().value_counts().keys())[:15]
all_markers = list(Line2D.markers.keys())[5:]
all_colors = sns.color_palette("colorblind", len(paper))
for j in range(len(paper)):    
    papermask = np.array([np.isin(paper[j],kpcdf.loc[i,'Paper(s)'].split(" ; ")).item() for i in kpcdf.index.values])
    for k in kpcdf[papermask].index.values:
        ulabel = paper[j] if k==kpcdf[papermask].index.values[0] else None
        plt.scatter(kpcdf[papermask]['z1'][k],kpcdf[papermask]['Sep(kpc)'][k],marker=all_markers[j],c=all_colors[j],s=30,label=ulabel)

# plot points with confidence >0.5 and have been imaged
confidence_mask = subkpc_dual['ST1 Confidence Flag']>=0.5
imaging_mask  = subkpc_dual['Parsed Analysis Method'].str.contains("Imaging")
distance_mask = subkpc_dual['Sep(kpc)']>0.2  # to remove radio sources
df = subkpc_dual[confidence_mask&imaging_mask&distance_mask]
lagn = np.log10([0.033e44,0.135e44,3e44,10**43.23*600,6e46])
subkpc_markers = list(Line2D.markers.keys())[1:6]
for i,m,mrk in zip(df.index.to_list(),lagn,subkpc_markers): # mark confirmed and very sure candidates differently
    wrapped_label = "\n".join(textwrap.wrap(df['Paper(s)'][i], width=60))
    sca = ax.scatter(df['z1'][i], df['Sep(kpc)'][i],label=wrapped_label,s=30,cmap='magma',c=m,marker=mrk,vmin=np.min(lagn)-0.5,vmax=np.max(lagn)+1)

cbar_ax = fig.add_axes([0.99, 0.125, 0.01, 0.75]) 
cbar = fig.colorbar(sca,cax=cbar_ax)
cbar.set_label("Log($L_{AGN}$) [erg/s]")

ax.set_xlabel("Redshift")
ax.set_ylabel("Projected separation [kpc]")
ax.set_xlim((-0.01,3))
ax.set_ylim((0.2,110))
# set top xlabel to look back time
ax_top = ax.secondary_xaxis("top")
ax_top.set_xlabel("Lookback time [Gyr]")
# interpolate to get tick positions for round lookback time
spl = CubicSpline(cosmo.lookback_time(np.linspace(0,0.5)),np.linspace(0,0.5))
xtick_pos = spl(np.arange(1,12,2))
ax_top.set_xticks(xtick_pos)
ax_top.set_xticklabels(np.arange(1,12,2))
ax.set_yscale('log')

ax.legend(ncol=3,fontsize=8,loc='lower center')
ax.grid(linestyle='--',alpha=0.5)
fig.tight_layout()
fig.savefig("hst.png",dpi=500);

compare OIII and IR for JA

In [ ]:
hst_sdss152 = pd.read_pickle("/home/insepien/research-data/alpaka/snap/hstP2_snap151_alpakaWithSDSS.pkl")
# remove 4 observed targets
observed_mask = hst_sdss152['Desig'].isin(['J1000+1242', 'J1010+1413','J1352+6541', 'J1356+1026','J1222-0007'])
hst_sdss148 = hst_sdss152[~observed_mask]
hst_sdss148.reset_index(inplace=True,drop=True)
print(f"final sample shape: {hst_sdss148.shape}")
snap_mask = (alpaka['RA'].isin(hst_sdss148['RA'])) & (alpaka['DEC'].isin(hst_sdss148['DEC']))


In [ ]:
fig,ax = plt.subplots(1,2,figsize=(10,4),sharex=True,dpi=300)
zm = alpaka['Z'] < 0.2
ax[0].scatter(alpaka['OIII_5007_LUM'], alpaka['wiseLum'],s=2,alpha=0.1,color='grey')
ax[1].scatter(alpaka['OIII_5007_LUM_DERRED'] ,alpaka['wiseLum'],s=2,alpha=0.1,color='grey')

#magellan
ax[0].scatter(magel['OIII_5007_LUM'], magel['wiseLum'],s=5,color='b',marker='x',label='Magellan')
ax[1].scatter(magel['OIII_5007_LUM_DERRED'] ,magel['wiseLum'],s=5,color='b',marker='x')

# snap
ax[0].scatter(alpaka['OIII_5007_LUM'][snap_mask], alpaka['wiseLum'][snap_mask],s=2,alpha=0.5,color='green',label='SNAP')
ax[1].scatter(alpaka['OIII_5007_LUM_DERRED'][snap_mask],alpaka['wiseLum'][snap_mask],s=2,alpha=0.5,color='green')

# j1010
# j1010mask = alpaka['Desig'] == "J1010+1413"
# j1000mask  = alpaka['Desig'] == "J1000+1242"
# [ax[0].scatter(alpaka['OIII_5007_LUM'][jmask], alpaka['wiseLum'][jmask],s=10,alpha=0.5,color='red',marker='x') for jmask in [j1000mask,j1010mask]]

[a.set_xscale('log') for a in ax]
[a.set_yscale('log') for a in ax]
[a.set_ylabel("$L_{\\rm{IR}}$") for a in ax]
[a.set_xlabel(lab) for a,lab in zip(ax,['$L_{\\rm{[OIII]}}$','$L_{\\rm{[OIII],dered}}$'])]
[a.set_xlim(1e40,1e44) for a in ax]
[a.set_ylim(3e42,1e46) for a in ax]
ax[0].legend(fontsize=10,loc='lower right')
fig.tight_layout();

# try resampling to correct incomplete lum

In [ ]:
# get all type 2 agn in z cut from mul with match in wise
type2 = alpaka[(alpaka['Z'] > 0.14) & (alpaka['Z'] < 0.22) & (alpaka['AGN_TYPE']==2)]
magel_withwise = type2[type2['desig'].isin(magel['desig'])]
# get histograms, using sqrt smaller sample as numbers of bin
lbol_all = np.log10(type2['irbol'])
bin = np.linspace(lbol_all.min(),lbol_all.max(),int(np.ceil(np.sqrt(39))))
hist_ful = plt.hist(lbol_all,label="mullaney type-2 agn \nmatched with WISE",bins=bin)
hist_magel = plt.hist(np.log10(np.concatenate([irbol,irbol_dual])),label='Magellan sample',bins=bin)
plt.yscale('log')
plt.xlabel('Log(L_bol)')
plt.ylabel("number of AGN")
plt.legend();

In [ ]:
# interpolate pdf for pretty plotting
binmid = (bin[:-1]+bin[1:])*0.5
pdf_full = CubicSpline(binmid,hist_ful[0])
pdf_magel = CubicSpline(binmid,hist_magel[0])
# use full sample pdf as weight
w = hist_ful[0]/np.sum(hist_ful[0])
# assign weight to each magel target
binnum = np.digitize(np.log10(magel_withwise['irbol']),bin)-1
weights = [w[b] for b in binnum]
magel_withwise['weights'] = weights
# sample magel with weights
new_sample_size = 20
magel_sub = [magel_withwise.sample(n=new_sample_size,weights='weights') for i in range(1000)]
pdfs_magel = [np.histogram(np.log10(magel_sub[i]['irbol']),bins=bin)[0] for i in range(1000)]
# random sample from full sample
pdfs = [np.histogram(np.log10(type2.sample(n=new_sample_size)['irbol']),bins=bin)[0] for i in range(1000)]

In [ ]:
maxind = np.argmax(hist_magel[0])
magel_corrected = w/w[maxind]*hist_magel[0]
magel_corrected_spl = CubicSpline(binmid,magel_corrected)

In [ ]:
# for pretty plot
x = np.linspace(binmid.min(),binmid.max(),20)
# plot full sample
plt.plot(x,pdf_full(x),c='b')
plt.scatter(binmid,hist_ful[0],c='b',label='full sample')
# plot magellan sample
plt.scatter(binmid,hist_magel[0],label='magel',c='r')
plt.plot(x,pdf_magel(x),c='r')
# correct by largest bin
plt.scatter(binmid,magel_corrected,c="g",label='corrected')
plt.plot(x,magel_corrected_spl(x),c='g')
# plot subsamples
[plt.plot(binmid, pdfs_magel[i],c='r',alpha=0.01) for i in range(500)];
[plt.plot(binmid, pdfs[i],c='b',alpha=0.01) for i in range(500)];

plt.plot(binmid[maxind], magel_corrected[maxind],c="r",marker="*",markersize=20)
plt.ylim(bottom=0.5)
plt.yscale('log')
plt.xlabel('Log(L_bol)')
plt.ylabel("number of AGN")
plt.legend(bbox_to_anchor=(1,1));

### malmquist correction
tried volume limited sample but our sample is already very bright, so doesn't work

In [ ]:
from astropy.cosmology import WMAP9 as cosmo
mlim = 22.2 # r-band
sdss = pd.read_pickle("/home/insepien/research-data/alpaka/sdss-cat/sdss_rband_171.pkl")
sdss40 = sdss[sdss['DESIG'].isin(mul['desig'])]
sdss40['r-mag'] = 22.5 - 2.5 * np.log10(sdss40['spectroFlux_r'])
sdss40 = pd.merge(sdss40,mul.rename(columns={'desig':'DESIG'})[['Z','DESIG']],on='DESIG')
sdss40['M'] = sdss40['r-mag']-5*np.log10((cosmo.angular_diameter_distance(sdss40['Z'])*u.Mpc/(10*u.pc).to(u.Mpc)))

Mlim = -16.5
mask = sdss40['M'] < Mlim
dmax = (10**((mlim-Mlim)/5)*10*u.pc).to(u.Mpc)
from scipy.interpolate import CubicSpline
spl = CubicSpline(cosmo.angular_diameter_distance(np.linspace(0,1,20)), np.linspace(0,1,20))
zmax = spl(dmax.value)
zmax

make table of dual fraction from literature

In [ ]:
dfr = pd.read_csv("/home/insepien/research-data/pop-result/lit_rev_frac/dualfrac_rev2.csv")
dfr_agn = dfr[~ dfr['lower dual frac'].isna()]
errs = []
not_nan_ind = dfr_agn['lower dual frac error'].dropna().index.values
nan_ind = dfr_agn.index[~ np.isin(dfr_agn.index,dfr_agn['lower dual frac error'].dropna().index.values)]
for i in range(len(dfr_agn)):
    if i in not_nan_ind:
        try:
            errs.append(np.abs(dfr_agn['lower dual frac'][i]-sorted(np.array(dfr_agn['lower dual frac error'][i].split(','),dtype=float))))
        except:
                print(i)
    else:
         errs.append(np.array([np.nan,np.nan]))
errs = pd.DataFrame(errs)

fmt = lambda minv, maxv: f"${minv:.0f}-{maxv:.0f}$"
fmt2 = lambda minv, maxv: f"${minv:.2f}-{maxv:.2f}$"
fmt_err = lambda val, low_err, up_err: f"${val:.3f}^{{-{low_err:.4f}}}_{{+{up_err:.4f}}}$"
fmt_lbol = lambda min,max : f"$10^{{{min:.0f}}}-10^{{{max:.0f}}}$"


redshift = [fmt2(minvl,maxvl) for minvl,maxvl in zip(dfr_agn['min z'],dfr_agn['max z'])]
sep = [fmt(minvl,maxvl) for minvl,maxvl in zip(dfr_agn['Min sep'],dfr_agn['Max sep'])]
lbols = [fmt_lbol(minvl,maxvl) for minvl,maxvl in zip(dfr_agn['min Lbol'],dfr_agn['max Lbol'])]
fracs = [fmt_err(val,errl,erru) for val,errl,erru in zip(dfr_agn['lower dual frac'],errs[0],errs[1])]

keys = ['Paper Name', 'Red shift', 'Separation $[kpc]$', "$L_{bol}~[erg~s^{-1}$]", "Selection method", "Dual fraction", "Fraction definition"]
tabb=pd.DataFrame([list(dfr_agn['Paper']),redshift,sep,lbols,list(dfr_agn['selection']),fracs,list(dfr_agn['note'])],index=keys).T.to_latex(column_format='c|c|c|c|c|c',index=False)
print(tabb)
